# Лабораторная работа №5
## Распознавание объектов на фотографиях
## Группа: БФИ1901

### Цель работы:
Реализовать распознавание объектов на фотографиях (Object Recognition in Photographs) CIFAR-10 (классификация небольших изображений по десяти классам: самолет, автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

### Задание:
* Ознакомиться со сверточными нейронными сетями
* Изучить построение модели в Keras в функциональном виде
* Изучить работу слоя разреживания (Dropout)

### Ход работы:
#### Подключение зависимостей и инициализация переменных
Подключим необходимые нам библиотеки:

In [ ]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

Создадим переменные, которые будут использоваться в программе:

In [ ]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
hidden_size = 512 # the dense layer will have 512 neurons

#### Импорт и преобразование данных
Загрузим данные CIFAR-10 и модифицируем их для использования в процессе обучения.

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels

Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

#### Конфигурация и запуск модели
Дальше создадим структуру нашей модели, запустим процесс обучения и потом протестируем модель.

In [ ]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
optimizer='adam', # using the Adam optimiser
metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...

batch_size=batch_size, nb_epoch=num_epochs, verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

#### Вопрос 1: Построить и обучить сверточную нейронную сеть
Ниже представлен полный код программы, по которому можно проверить результат работы нашей сети.

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
hidden_size = 512 # the dense layer will have 512 neurons

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)  # Train the model using the training set holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

2022-03-22 16:49:39.483829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 16:49:39.483862: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-22 16:49:44.744132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 16:49:44.745491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 16:49:44.745637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

Epoch 1/200
1400/1407 [============================>.] - ETA: 0s - loss: 1.9122 - accuracy: 0.2735

2022-03-22 16:49:56.073184: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 61440000 exceeds 10% of free system memory.


1407/1407 [==============================] - 10s 7ms/step - loss: 1.9111 - accuracy: 0.2739 - val_loss: 1.6735 - val_accuracy: 0.3728
Epoch 2/200
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6886 - accuracy: 0.3718 - val_loss: 1.5506 - val_accuracy: 0.4214
Epoch 3/200
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6180 - accuracy: 0.3993 - val_loss: 1.5119 - val_accuracy: 0.4374
Epoch 4/200
1407/1407 [==============================] - 9s 7ms/step - loss: 1.5700 - accuracy: 0.4228 - val_loss: 1.4487 - val_accuracy: 0.4640
Epoch 5/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.5286 - accuracy: 0.4394 - val_loss: 1.3987 - val_accuracy: 0.4994
Epoch 6/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.4996 - accuracy: 0.4522 - val_loss: 1.3585 - val_accuracy: 0.5050
Epoch 7/200
1407/1407 [==============================] - 9s 7ms/step - loss: 1.4796 - accuracy: 0.4598 - val_loss: 1.3599 - val_accuracy: 0

1407/1407 [==============================] - 8s 6ms/step - loss: 1.1983 - accuracy: 0.5770 - val_loss: 1.2187 - val_accuracy: 0.5644
Epoch 114/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1989 - accuracy: 0.5746 - val_loss: 1.1956 - val_accuracy: 0.5790
Epoch 115/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1934 - accuracy: 0.5775 - val_loss: 1.2136 - val_accuracy: 0.5690
Epoch 116/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1961 - accuracy: 0.5760 - val_loss: 1.1910 - val_accuracy: 0.5826
Epoch 117/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1992 - accuracy: 0.5733 - val_loss: 1.2108 - val_accuracy: 0.5698
Epoch 118/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1880 - accuracy: 0.5772 - val_loss: 1.2008 - val_accuracy: 0.5802
Epoch 119/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.1976 - accuracy: 0.5780 - val_loss: 1.2290 - val_ac

Epoch 169/200
1407/1407 [==============================] - 9s 7ms/step - loss: 1.1702 - accuracy: 0.5852 - val_loss: 1.2042 - val_accuracy: 0.5702
Epoch 170/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1727 - accuracy: 0.5840 - val_loss: 1.2008 - val_accuracy: 0.5754
Epoch 171/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1667 - accuracy: 0.5889 - val_loss: 1.1949 - val_accuracy: 0.5738
Epoch 172/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1684 - accuracy: 0.5845 - val_loss: 1.1880 - val_accuracy: 0.5832
Epoch 173/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1704 - accuracy: 0.5855 - val_loss: 1.1852 - val_accuracy: 0.5792
Epoch 174/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1681 - accuracy: 0.5862 - val_loss: 1.1920 - val_accuracy: 0.5826
Epoch 175/200
1407/1407 [==============================] - 9s 6ms/step - loss: 1.1686 - accuracy: 0.5862 - val_loss: 1

2022-03-22 17:19:09.259258: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 3ms/step - loss: 796.0125 - accuracy: 0.1588


[796.012451171875, 0.15880000591278076]

#### Вопрос 2: Исследовать работу сеть без слоя Dropout
Уберем из нашей модели слои Dropout и повторно запустим обучение, чтобы посмотреть на результат.

In [2]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
hidden_size = 512 # the dense layer will have 512 neurons

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(pool_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(pool_2)
hidden = Dense(hidden_size, activation='relu')(flat)
out = Dense(num_classes, activation='softmax')(hidden)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)  # Train the model using the training set holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200


2022-03-22 14:19:48.167815: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 552960000 exceeds 10% of free system memory.


1401/1407 [============================>.] - ETA: 0s - loss: 1.7991 - accuracy: 0.3194

2022-03-22 14:19:55.972870: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 61440000 exceeds 10% of free system memory.


1407/1407 [==============================] - 8s 5ms/step - loss: 1.7984 - accuracy: 0.3198 - val_loss: 1.6983 - val_accuracy: 0.3716
Epoch 2/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.5612 - accuracy: 0.4218 - val_loss: 1.5370 - val_accuracy: 0.4384
Epoch 3/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4654 - accuracy: 0.4596 - val_loss: 1.4380 - val_accuracy: 0.4664
Epoch 4/200
1407/1407 [==============================] - 8s 5ms/step - loss: 1.3959 - accuracy: 0.4858 - val_loss: 1.3971 - val_accuracy: 0.4878
Epoch 5/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.3448 - accuracy: 0.5090 - val_loss: 1.3282 - val_accuracy: 0.5100
Epoch 6/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.2972 - accuracy: 0.5282 - val_loss: 1.3298 - val_accuracy: 0.5152
Epoch 7/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.2624 - accuracy: 0.5406 - val_loss: 1.2842 - val_accuracy: 0.52

1407/1407 [==============================] - 9s 6ms/step - loss: 0.4435 - accuracy: 0.8364 - val_loss: 2.6454 - val_accuracy: 0.5214
Epoch 114/200
1407/1407 [==============================] - 9s 6ms/step - loss: 0.4440 - accuracy: 0.8351 - val_loss: 2.8202 - val_accuracy: 0.5270
Epoch 115/200
1407/1407 [==============================] - 8s 6ms/step - loss: 0.4433 - accuracy: 0.8345 - val_loss: 2.8987 - val_accuracy: 0.5126
Epoch 116/200
1407/1407 [==============================] - 9s 6ms/step - loss: 0.4463 - accuracy: 0.8323 - val_loss: 2.8036 - val_accuracy: 0.5202
Epoch 117/200
1407/1407 [==============================] - 8s 6ms/step - loss: 0.4395 - accuracy: 0.8366 - val_loss: 2.7780 - val_accuracy: 0.5154
Epoch 118/200
1407/1407 [==============================] - 8s 5ms/step - loss: 0.4291 - accuracy: 0.8402 - val_loss: 2.7345 - val_accuracy: 0.5100
Epoch 119/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.4389 - accuracy: 0.8376 - val_loss: 2.9226 - val_ac

Epoch 169/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.3865 - accuracy: 0.8599 - val_loss: 3.4924 - val_accuracy: 0.5070
Epoch 170/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.3833 - accuracy: 0.8603 - val_loss: 3.6477 - val_accuracy: 0.5176
Epoch 171/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.3867 - accuracy: 0.8586 - val_loss: 3.6575 - val_accuracy: 0.5042
Epoch 172/200
1407/1407 [==============================] - 8s 5ms/step - loss: 0.3738 - accuracy: 0.8625 - val_loss: 3.5409 - val_accuracy: 0.5112
Epoch 173/200
1407/1407 [==============================] - 8s 5ms/step - loss: 0.3903 - accuracy: 0.8575 - val_loss: 3.6071 - val_accuracy: 0.5230
Epoch 174/200
1407/1407 [==============================] - 8s 5ms/step - loss: 0.3564 - accuracy: 0.8701 - val_loss: 3.5691 - val_accuracy: 0.5084
Epoch 175/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.3920 - accuracy: 0.8573 - val_loss: 3

2022-03-22 14:46:59.138567: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 3ms/step - loss: 2236.3215 - accuracy: 0.2771


[2236.321533203125, 0.27709999680519104]

Таким образом, удаление слоев Dropout позитивно сказалось на точности во время обучения и тестирования.

#### Вопрос 3: Исследовать работу сети при разных размерах ядра свертки
Увеличим количество ядер свертки в два раза. Запустим обучение, чтобы посмотреть результат.

In [2]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 64 # we will initially have 64 kernels per conv. layer...
conv_depth_2 = 128 # ...switching to 128 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
hidden_size = 512 # the dense layer will have 512 neurons

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)  # Train the model using the training set holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200


2022-03-22 14:59:26.836545: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 552960000 exceeds 10% of free system memory.


1407/1407 [==============================] - 13s 9ms/step - loss: 1.8440 - accuracy: 0.3022 - val_loss: 1.5752 - val_accuracy: 0.4174
Epoch 2/200
1407/1407 [==============================] - 13s 9ms/step - loss: 1.6119 - accuracy: 0.4066 - val_loss: 1.5139 - val_accuracy: 0.4438
Epoch 3/200
1407/1407 [==============================] - 13s 10ms/step - loss: 1.5259 - accuracy: 0.4419 - val_loss: 1.4176 - val_accuracy: 0.4786
Epoch 4/200
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4654 - accuracy: 0.4652 - val_loss: 1.3294 - val_accuracy: 0.5240
Epoch 5/200
1407/1407 [==============================] - 14s 10ms/step - loss: 1.4182 - accuracy: 0.4880 - val_loss: 1.3228 - val_accuracy: 0.5102
Epoch 6/200
1407/1407 [==============================] - 13s 9ms/step - loss: 1.3789 - accuracy: 0.5029 - val_loss: 1.3180 - val_accuracy: 0.5226
Epoch 7/200
1407/1407 [==============================] - 14s 10ms/step - loss: 1.3477 - accuracy: 0.5133 - val_loss: 1.2460 - val_acc

1407/1407 [==============================] - 14s 10ms/step - loss: 0.8631 - accuracy: 0.6990 - val_loss: 1.1571 - val_accuracy: 0.6094
Epoch 112/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.8671 - accuracy: 0.6959 - val_loss: 1.1506 - val_accuracy: 0.6116
Epoch 113/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.8600 - accuracy: 0.7004 - val_loss: 1.1436 - val_accuracy: 0.5992
Epoch 114/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.8599 - accuracy: 0.6997 - val_loss: 1.1315 - val_accuracy: 0.6142
Epoch 115/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.8580 - accuracy: 0.7026 - val_loss: 1.1574 - val_accuracy: 0.6066
Epoch 116/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.8534 - accuracy: 0.7039 - val_loss: 1.1620 - val_accuracy: 0.6062
Epoch 117/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.8676 - accuracy: 0.6994 - val_loss: 1.1402 

[1006.521728515625, 0.2011999934911728]

Уменьшим количество ядер свертки в два раза. Запустим обучение, чтобы посмотреть результат.

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 16 # we will initially have 64 kernels per conv. layer...
conv_depth_2 = 32 # ...switching to 128 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
hidden_size = 512 # the dense layer will have 512 neurons

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)  # Train the model using the training set holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

2022-03-22 15:52:43.616822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 15:52:43.616874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-22 15:52:48.564288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 15:52:48.565527: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-22 15:52:48.565671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

Epoch 1/200
1406/1407 [============================>.] - ETA: 0s - loss: 1.9738 - accuracy: 0.2543

2022-03-22 15:52:57.046346: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 61440000 exceeds 10% of free system memory.


1407/1407 [==============================] - 8s 5ms/step - loss: 1.9738 - accuracy: 0.2543 - val_loss: 1.7395 - val_accuracy: 0.3580
Epoch 2/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.7912 - accuracy: 0.3331 - val_loss: 1.6728 - val_accuracy: 0.3764
Epoch 3/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.7303 - accuracy: 0.3565 - val_loss: 1.6016 - val_accuracy: 0.4200
Epoch 4/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6978 - accuracy: 0.3711 - val_loss: 1.5852 - val_accuracy: 0.4176
Epoch 5/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6728 - accuracy: 0.3822 - val_loss: 1.5438 - val_accuracy: 0.4372
Epoch 6/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6497 - accuracy: 0.3925 - val_loss: 1.5263 - val_accuracy: 0.4418
Epoch 7/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6348 - accuracy: 0.3992 - val_loss: 1.5007 - val_accuracy: 0.45

1407/1407 [==============================] - 7s 5ms/step - loss: 1.4428 - accuracy: 0.4805 - val_loss: 1.3865 - val_accuracy: 0.5148
Epoch 114/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4480 - accuracy: 0.4771 - val_loss: 1.4094 - val_accuracy: 0.5054
Epoch 115/200
1407/1407 [==============================] - 6s 5ms/step - loss: 1.4402 - accuracy: 0.4812 - val_loss: 1.4031 - val_accuracy: 0.5078
Epoch 116/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4445 - accuracy: 0.4808 - val_loss: 1.3998 - val_accuracy: 0.5066
Epoch 117/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4433 - accuracy: 0.4779 - val_loss: 1.4107 - val_accuracy: 0.5018
Epoch 118/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4420 - accuracy: 0.4785 - val_loss: 1.4038 - val_accuracy: 0.5086
Epoch 119/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4439 - accuracy: 0.4796 - val_loss: 1.4015 - val_ac

Epoch 169/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4299 - accuracy: 0.4865 - val_loss: 1.3980 - val_accuracy: 0.5126
Epoch 170/200
1407/1407 [==============================] - 8s 5ms/step - loss: 1.4295 - accuracy: 0.4854 - val_loss: 1.4117 - val_accuracy: 0.5102
Epoch 171/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.4301 - accuracy: 0.4838 - val_loss: 1.3959 - val_accuracy: 0.5164
Epoch 172/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4345 - accuracy: 0.4851 - val_loss: 1.3757 - val_accuracy: 0.5142
Epoch 173/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4308 - accuracy: 0.4862 - val_loss: 1.3852 - val_accuracy: 0.5122
Epoch 174/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4310 - accuracy: 0.4837 - val_loss: 1.3904 - val_accuracy: 0.5136
Epoch 175/200
1407/1407 [==============================] - 7s 5ms/step - loss: 1.4267 - accuracy: 0.4865 - val_loss: 1

2022-03-22 16:16:12.665604: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 3ms/step - loss: 304.8032 - accuracy: 0.2290


[304.80322265625, 0.2290000021457672]

Таким образом, увеличивая число ядер свертки мы улучшаем точность в процессе обучения, однако при тестировании модели результат остается без значительных изменений.

#### Вывод
Мы построили и обучили сверточную нейронную сеть, протестировали как влияет размерность ядра свертки на обучение и тестирование модели, обучили модель без слоев Dropout.